# Sentiment Analysis Overview
* sentiment is a measure of how positive or negative something is
* we are going to build a very simple sentiment analyzer 
* These are amazon reviews, so they come with 5 star ratings, and we are going to look at the electronics category 
* These are XML files, so we will need an XML parser 

# NLP Terminology
* **Corpus**: Collection of text
* **Tokens**: Words and punctuation that make up the corpus. 
* **Type**: a distinct token. Ex. "Run, Lola Run" has four tokens (comma counts as one) and 3 types.
* **Vocabulary**: The set of all types. 
* The google corpus (collection of text) has 1 trillion tokens, and only 13 million types. English only has 1 million dictionary words, but the google corpus includes types such as "www.facebook.com". 

# Outline 
* we are just going to look at the electronics category 
* we could use the 5 star targets to do regression, but lets just do classification since they are already marked "positive" and "negative"
* XML parser (going to use BeautifulSoup)
* only look at **review_test**
* To create our feature vector, we will count up the number of occurences of each word, and divided it by the total number of words  
* For that to work we will need two passes through the data
    1. One to collect the total number of distinct words, so that we know the size of our feature vector, in other words the vocabulary size, and possibly remove stop words like "this", "is", "I", "to", etc, to decrease the vocabulary size. The goal here is to know the index of each token
    2. On the second pass, we will be able to assign values to each data vector which index corresponds to which words, and one to create data vectors 
* once we have that, it is simply a matter of creating a classifier like the one we did for our spam detector 
* so we will use logistic regression, so we can intepret the weights! 
* Ex: if you see a word like horrible and it has a weight of minus 1, it is associated with negative reviews

---
# Sentiment Analysis in Python using Logistic Regression 

In [1]:
import nltk
import numpy as np

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

wordnet_lemmatizer = WordNetLemmatizer()                                # this turns words into their base form 

stopwords = set(w.rstrip() for w in open('data/stopwords.txt'))         # grab stop words 

positive_reviews = BeautifulSoup(open('data/electronics/positive.review').read(), "lxml")   # get pos reviews
positive_reviews = positive_reviews.findAll('review_text')                                  # only want rev text

negative_reviews = BeautifulSoup(open('data/electronics/negative.review').read(), "lxml")
negative_reviews = negative_reviews.findAll('review_text')

### Class imbalance 
There are more positive than negative reviews, so we are going to shuffle the positive reviews and then cut off any extra that we may have so that they are both the same size. 

In [2]:
np.random.shuffle(positive_reviews)
positive_reviews = positive_reviews[:len(negative_reviews)]

### Tokenizer function
Lets now create a tokenizer function that can be used on our specific reviews.

In [3]:
def my_tokenizer(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)                        # essentially string.split()
    tokens = [t for t in tokens if len(t) > 2]                     # get rid of short words
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]     # get words to base form
    tokens = [t for t in tokens if t not in stopwords]
    return tokens

### Index each word
We now need to create an index for each of the words, so that each word has an index in the final data vector. However, to able able to do that we need to know the size of the final data vector, and to be able to know that we need to know how big the vocabulary is. Remember, the **vocabulary** is just the set of all types!

We are essentially going to look at every individual review, tokenize them, and then add those tokens 1 by 1 to the map if they do not exist yet.

In [4]:
word_index_map = {}                            # our vocabulary - dictionary that will map words to dictionaries
current_index = 0                              # counter increases whenever we see a new word

positive_tokenized = []
negative_tokenized = []

# --------- loop through positive reviews ---------
for review in positive_reviews:              
    tokens = my_tokenizer(review.text)          # converts single review into array of tokens (split function)
    positive_tokenized.append(tokens)
    for token in tokens:                        # loops through array of tokens for specific review
        if token not in word_index_map:                        # if the token is not in the map, add it
            word_index_map[token] = current_index          
            current_index += 1                                 # increment current index
                
# --------- loop through negative reviews ---------
for review in negative_reviews:              
    tokens = my_tokenizer(review.text)          
    negative_tokenized.append(tokens)
    for token in tokens:                       
        if token not in word_index_map:                        
            word_index_map[token] = current_index          
            current_index += 1                             

### Convert tokens into vector
Now that we have our tokens and vocabulary, we need to convert our tokens into a vector. Because we are going to shuffle our train and test sets again, we are going to want to put labels and vector into same array for now since it makes it easier to shuffle. 

Note, this function operates on **one** review. So the +1 is creating our label, and this function is basically designed to take our input vector from an english form the a numeric vector form.

In [5]:
def tokens_to_vector(tokens, label):
    xy_data = np.zeros(len(word_index_map) + 1)          # equal to the vocab size + 1 for the label 
    for t in tokens:                                     # loop through every token
        i = word_index_map[t]                            # get index from word index map
        xy_data[i] += 1                                  # increment data at that index 
    xy_data = xy_data / xy_data.sum()                    # divide entire array by total, so they add to 1
    xy_data[-1] = label                                  # set last element to label
    return xy_data

Time to actually assign these tokens to vectors.

In [6]:
N = len(positive_tokenized) + len(negative_tokenized)               # total number of examples 
data = np.zeros((N, len(word_index_map) + 1))                       # N examples x vocab size + 1 for label
i = 0                                                               # counter to keep track of sample

for tokens in positive_tokenized:                                   # loop through postive tokenized reviews
    xy = tokens_to_vector(tokens, 1)                                # passing in 1 because these are pos reviews
    data[i,:] = xy                                                  # set data row to that of the input vector
    i += 1                                                          # increment 1
    
for tokens in negative_tokenized:                                   
    xy = tokens_to_vector(tokens, 0)                                
    data[i,:] = xy                                                 
    i += 1         

Our data is now 1000 rows of positively labeled reviews, followed by 1000 rows of negatively labeled reviews. Lets shuffle before getting our train and test set.

In [7]:
np.random.shuffle(data)
X = data[:, :-1]
Y = data[:, -1]

Xtrain = X[:-100,]
Ytrain = Y[:-100,]
Xtest = X[-100:,]
Ytest = Y[-100:,]


model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print("Classification Rate: ", model.score(Xtest, Ytest))

Classification Rate:  0.78


## Classification Rate
We end up with a classification rate of 0.71, which is not so great, but it is better than random guessing. 

## Sentiment Analysis
Something interesting that we can do is look at the weights of each word, to see if that word has positive or negative sentiment. 

In [8]:
threshold = 0.7 
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight )

easy 1.7231721659895853
quality 1.4692546325806657
wa -1.5355800823087296
item -0.9826509302695917
little 0.8100985329147102
price 2.8283260263389614
speaker 0.8956714500170458
sound 1.007178880160086
n't -1.9457731877147226
doe -1.3582381733513151
excellent 1.4018492494647046
ha 0.753363606240854
highly 0.9810325307216232
perfect 0.9870088209284515
you 1.0856753465053635
fast 0.9043338378144629
buy -0.8707359259356201
pretty 0.7592510923698834
support -0.7804040442394683
waste -1.0126929064624308
've 0.8224985047728288
love 1.174896398784311
tried -0.7923810693766183
memory 0.9868637432403627
poor -0.792898589716643
money -1.0001433387300387
month -0.7739761734398145
then -1.1887336240919897
week -0.7105599384556058
return -1.1699173504176226
returned -0.823506888900688


---

<br>
# Appendix - Additional Metrics and Analysis
From here on out the rest of the analysis should be treated as an appendix. 

In [9]:
# What is the mean number of words in a positive review?
positive_reviews_length = np.zeros(len(positive_reviews))
for index, review in enumerate(positive_reviews):
  positive_reviews_length[index] = len(review.text.split(" "))

print("Mean number of words in positive reviews:", positive_reviews_length.mean())

Mean number of words in positive reviews: 110.07


In [10]:
len(word_index_map)

11088